In [4]:
import snowflake
from snowflake.snowpark.session import Session
import os
import pandas as pd
import numpy as np
import xgboost as xgb
import toml
import json

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import snowflake
from snowflake.ml.registry import Registry
from snowflake.ml._internal.utils import identifier
from snowflake.ml.model import model_signature
import snowflake.snowpark.types as T




In [5]:
config = toml.load('/Users/ebotwick/library/Application Support/snowflake/config.toml').get('connections').get('ebotwick_default')


session = Session.builder.configs(config).create()

In [6]:
session.use_database("ML_HOL_DB")
session.use_schema("ML_HOL_SCHEMA")
session.use_role("TEMP_HOL_USER")

In [7]:
session.sql('SHOW TABLES;').show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"               |"database_name"  |"schema_name"  |"kind"  |"comment"  |"cluster_by"  |"rows"  |"bytes"  |"owner"        |"retention_time"  |"automatic_clustering"  |"change_tracking"  |"search_optimization"  |"search_optimization_progress"  |"search_optimization_bytes"  |"is_external"  |"enable_schema_evolution"  |"owner_role_type"  |"is_event"  |"budget"  |"is_hybrid"  |"is_iceberg"  |"is_dynamic"  |"is_immutable"  |
------------------------------------------------------------------------

In [8]:
scoring_data = session.table('SCORING_DATA')

In [9]:
scoring_data.show(5)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CARAT"               |"DEPTH"              |"TABLE_PCT"          |"X"                  |"Y"                  |"Z"                  |"CUT_OE"  |"COLOR_OE"  |"CLARITY_OE"  |"COLOR"  |"CLARITY"  |"PRICE"  |"CUT"    |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.006237006237006237  |0.5138888888888888   |0.23076923076923073  |0.3677839851024209   |0.06757215619694397  |0.07641509433962265  |0.0       |1.0         |6.0           |E        |SI2        |326      |IDEAL    |
|0.002079002079002079  |0.46666666666666656  |0.34615384615384615  |0.36219739292364994  |0.06519524617996604  |0.07264150943396226  |1.

In [10]:
db = identifier._get_unescaped_name(session.get_current_database())
schema = identifier._get_unescaped_name(session.get_current_schema())
snowflake_ml_registry = Registry(session=session, database_name=db, schema_name=schema)

In [11]:
snowflake_ml_registry.show_models()

,created_on,name,model_type,database_name,schema_name,comment,owner,default_version_name,versions,aliases
0,2024-06-17 23:22:19.546000-07:00,DIAMONDS_PRICE_PREDICTION_EBOTWICK,USER_MODEL,ML_HOL_DB,ML_HOL_SCHEMA,None,TEMP_HOL_USER,V0,"[""V0"",""V1""]","{""DEFAULT"":""V0"",""FIRST"":""V0"",""LAST"":""V1""}"
1,2024-06-18 12:08:21.887000-07:00,DIAMONDS_PRICE_PREDICTION_EBOTWICK_3,USER_MODEL,ML_HOL_DB,ML_HOL_SCHEMA,None,TEMP_HOL_USER,V0,"[""V0"",""V1"",""V2"",""V3""]","{""DEFAULT"":""V0"",""FIRST"":""V0"",""LAST"":""V3""}"
2,2024-09-06 10:14:14.206000-07:00,MODEL_WITH_EXPLAIN_ENABLED,USER_MODEL,ML_HOL_DB,ML_HOL_SCHEMA,None,TEMP_HOL_USER,EXPLAIN_V0,"[""EXPLAIN_V0""]","{""DEFAULT"":""EXPLAIN_V0"",""FIRST"":""EXPLAIN_V0"",""..."


In [12]:
model = snowflake_ml_registry.get_model('DIAMONDS_PRICE_PREDICTION_EBOTWICK').version('V0')
model._functions

[{'name': 'PREDICT',
  'target_method': 'predict',
  'target_method_function_type': 'FUNCTION',
  'signature': ModelSignature(
                      inputs=[
                          FeatureSpec(dtype=DataType.DOUBLE, name='CUT_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='COLOR_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CLARITY_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CARAT'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='DEPTH'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='TABLE_PCT'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='X'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='Y'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='Z')
                      ],
                      outputs=[
                          FeatureSpec(dtype=DataType.DOUBLE, name='CUT_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='COLOR_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CLARITY_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CARAT'),
  		FeatureSpec(dtype=DataType

In [13]:
#Score loaded model version
result = model.run(scoring_data, function_name="predict")
result = result.rename({result.columns[-1]: "predicted_price"})


In [14]:
result.show(25)


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"COLOR"  |"CLARITY"  |"PRICE"  |"CUT"      |"CUT_OE"  |"COLOR_OE"  |"CLARITY_OE"  |"CARAT"               |"DEPTH"              |"TABLE_PCT"          |"X"                  |"Y"                  |"Z"                  |"PREDICTED_PRICE"   |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|E        |SI2        |326      |IDEAL      |0.0       |1.0         |6.0           |0.006237006237006237  |0.5138888888888888   |0.23076923076923073  |0.3677839851024209   |0.06757215619694397  |0.07641509433962265  |439.5343933105469   |
|E        |SI1        |326      |PREMIUM    

In [15]:
from snowflake.ml.modeling.metrics import mean_absolute_percentage_error, r2_score

mape = mean_absolute_percentage_error(df=result, 
                                        y_true_col_names="PRICE", 
                                        y_pred_col_names="PREDICTED_PRICE")

r2 = r2_score(df=result, 
                y_true_col_name="PRICE", 
                y_pred_col_name="PREDICTED_PRICE")

print("MAPE:", mape)
print("R2:", r2)

MAPE: 0.06520824113175573
R2: 0.9905417941107357


In [16]:
old_model = snowflake_ml_registry.get_model("DIAMONDS_PRICE_PREDICTION_EBOTWICK").default


In [19]:
model.load(force=True)

/opt/anaconda3/envs/snowflake-trulens-py39/lib/python3.9/site-packages/snowflake/ml/_internal/telemetry.py:418: RuntimeWarning: Loading model requires to have the exact the same environment as the one when logging the model, otherwise, the model might be not functional or some other problems might occur.
  res = func(*args, **kwargs)


In [22]:
old_model = snowflake_ml_registry.get_model("DIAMONDS_PRICE_PREDICTION_EBOTWICK").default
# model = mv_old.load()
mv_new = snowflake_ml_registry.log_model(
    model.load(force=True),
    model_name="model_with_explain_enabled",
    version_name="explain_v1",
    conda_dependencies=["snowflake-ml-python"],
    sample_input_data = scoring_data
)

/opt/anaconda3/envs/snowflake-trulens-py39/lib/python3.9/contextlib.py:119: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/opt/anaconda3/envs/snowflake-trulens-py39/lib/python3.9/site-packages/snowflake/ml/model/_packager/model_packager.py:92: UserWarning: Inferring model signature from sample input or providing model signature for Snowpark ML Modeling model is not required. Model signature will automatically be inferred during fitting. 
  handler.save_model(


In [26]:
mv_new.show_functions()

[{'name': 'PREDICT',
  'target_method': 'predict',
  'target_method_function_type': 'FUNCTION',
  'signature': ModelSignature(
                      inputs=[
                          FeatureSpec(dtype=DataType.DOUBLE, name='CUT_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='COLOR_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CLARITY_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CARAT'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='DEPTH'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='TABLE_PCT'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='X'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='Y'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='Z')
                      ],
                      outputs=[
                          FeatureSpec(dtype=DataType.DOUBLE, name='CUT_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='COLOR_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CLARITY_OE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CARAT'),
  		FeatureSpec(dtype=DataType

In [27]:
mv_new.run(scoring_data, function_name="explain").show(5)

ValueError: (0000) There is no method with name explain available in the model ML_HOL_DB.ML_HOL_SCHEMA.MODEL_WITH_EXPLAIN_ENABLED version EXPLAIN_V1